# Getting started

## Prerequisites

### Installation

This tutorial requires **signac**, so make sure to install the package before starting.
The easiest way to do so is using conda:

```$ conda install -c glotzer signac```

or pip:

```pip install signac --user```

Please refer to http://signac.readthedocs.io/en/latest/installation.html#installation for detailed instructions on how to install signac.

## A minimal example

For this tutorial we want to compute the volume of an ideal gas as a function of its pressure and temperature.

**pV=NkT**

In [35]:
def V_idg(N, p, kT):
    return N * kT / p

We can execute the complete study in just a few lines of code.
First, we enter the project directory and get a project handle:

In [36]:
import signac

% cd projects/ideal-gas-example
project = signac.get_project()

[Errno 2] No such file or directory: 'projects/ideal-gas-example'
/Users/csadorf/local/signac-examples/projects/ideal-gas-example


Then we iterate over different state points and perform the ideal gas calculation:

In [46]:
for p in 0.1, 1.0, 10.0:
    sp = {'p': p, 'kT': 1.0, 'N': 1000}
    job = project.open_job(sp)
    job.document['V'] = V_idg(**sp)

We iterate over the variable of interest *p* and construct a complete state point *sp* which contains all the meta data associated with our data.
In this simple example the meta data is very compact, but in principle the state point may be highly complex.

Next, we obtain a *job* handle and store the result of the calculation with in the *job document*.
The *job document* is a persistent dictionary for storage of simple key-value pairs.
Here, we exploit that the state point dictionary *sp* can easily be passed into the `V_idg()` function using the keyword expansion syntax (`**sp`).

We can then examine our results by iterating over the data space:

In [38]:
for job in project.find_jobs():
    print(job.statepoint()['p'], job.document['V'])

10.0 100.0
1.0 1000.0
0.1 10000.0


## The Basics

In the minimal example we initialized the data space implicitly. Let’s see how we can initialize it explicitly. In general, the data space needs to contain all parameters that will affect our data. For the ideal gas that is a 3-dimensional space spanned by the temperature *T*, the pressure *p* and the system size *N*.

Each state point represents a unique set of parameters that we want to associate with data. In terms of **signac** this relationship is represented by a *job*.
We use the `open_job()` method to get a handle on the *job dataspace*:

In [39]:
job = project.open_job({'p': 1.0, 'kT': 1.0, 'N': 1000})

The *job* handle tightly couples our input parameters (*p*, *T*, *N*) with the storage location of the output data.
You can inspect both the input parameters and the storage location:

In [40]:
print(job)
print(job.statepoint())
print(job.workspace())

ee617ad585a90809947709a7a45dda9a
{'p': 1.0, 'kT': 1.0, 'N': 1000}
/Users/csadorf/local/signac-examples/projects/ideal-gas-example/workspace/ee617ad585a90809947709a7a45dda9a


We can use this to store data, which is safely associated with the input parameters.

In [47]:
# Get the job handle for the input variables
job = project.open_job({'p': 1.0, 'kT': 1.0, 'N': 1000})
# Calculate the result
V = V_idg(** job.statepoint())

# The following two methods for storing the result are equivalent:

# Use job.fn() to generate a statepoint-specific filename.
with open(job.fn('V.txt'), 'w') as file:
    file.write(str(V) + '\n')
 
# Use the job handle as context manager to enter the job workspace directory
# during the duration of the operation.
with job:
    with open('V.txt', 'w') as file:
        file.write(str(V) + '\n')

In the above example the data was stored in a file called `V.txt`.
Light-weight data such as simple key-value pairs can alternatively stored in the *job document* as was shown in the minimal example.

The *job document* is a persistent dictionary to store job-associated data.
The data is safely stored within a JSON dictionary.

In [ ]:
# Get the job handle for the input variables
job = project.open_job({'p': 1.0, 'kT': 1.0, 'N': 1000})
# Calculate the result
V = V_idg(** job.statepoint())

job.document['V'] = V

Usually, we are interested in more than one state point, in which case we can simply iterate over the variables of interest.

In [ ]:
for pressure in 0.1, 1.0, 10.0:
    statepoint = {'p': pressure, 'T': 10.0, 'N': 1000}
    job = project.open_job(statepoint)
    job.document['V'] = V_idg(** job.statepoint())
    print(job, 'initialized')

In [22]:
for pressure in 0.1, 1.0, 10.0:
    statepoint = {'p': pressure, 'T': 10.0, 'N': 1000}
    job = project.open_job(statepoint)
    job.init()
    print(job, 'initialized')

8af3e80c0037fd3227ab2bdc1adb5daa initialized
b44c2480075402119de5abf2831316a1 initialized
9ef1a35d0fcf3d8eb8af99ace89b37db initialized


The output shows the *job ids* associated with each state point.
The *job id* is a unique identifier representing the state point.
Typical computational studies require vastly more parameters than the three we need for the ideal gas computation.
Especially in those cases the *job id* is a much more compact representation of the whole state point.

As we did not explicitly specify the location of our project’s workspace it defaulted to projects/ideal-gas-example/workspace. The project’s workspace has been populated with directories for each state point:

In [23]:
import os

os.listdir('workspace')

['8af3e80c0037fd3227ab2bdc1adb5daa',
 '9ef1a35d0fcf3d8eb8af99ace89b37db',
 'b44c2480075402119de5abf2831316a1']

Let's go ahead and perform our computation.
We define two different functions to highlight the important concept of *operations*.

The first function `calc_volume` generates the data for a set of input parameters, the second function `compute_volume()` stores the data within the *project workspace*.

In [24]:
def calc_volume(N, T, p):
    "Compute the volume of an ideal gas."
    return N * T / p

def compute_volume(job):
    "Compute the volume of this state point."
    sp = job.statepoint()
    with job:
        V = calc_volume(sp['N'], sp['T'], sp['p'])
        with open('V.txt', 'w') as file:
            file.write(str(V)+'\n')
        print(job, 'computed volume')